# Makale Öneri Sistemi | Content-Based Recommendation

Makale içeriklerine dayalı **içerik temelli öneri sistemi**. TF-IDF vektörleştirme ve Cosine Similarity kullanılarak her makale için benzer içerikler önerilmektedir.

| Özellik | Detay |
|---------|-------|
| **Veri Seti** | articles.csv — 34 makale, 2 sütun (article, title) |
| **Problem Türü** | İçerik Tabanlı Filtreleme (Gözetimsiz) |
| **Yöntem** | TF-IDF Vektörleştirme + Cosine Similarity |
| **Çıktı** | Her makale için en benzer 4 makale önerisi |

**İş Akışı:** Veri Yükleme → Metin Temizleme → TF-IDF Dönüşümü → Cosine Similarity Matrisi → Öneri Fonksiyonu → Kayıt

In [ ]:
# Temel Kütüphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Makine Öğrenmesi
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
import joblib

print('✅ Kütüphaneler yüklendi.')


## 📊 1. Veri Yükleme ve İlk İnceleme

In [17]:


try:
    data = pd.read_csv('articles.csv', encoding='utf-8')
except:
    data = pd.read_csv('articles.csv', encoding='latin1')

if 'df' in locals():
    print('--- İlk 5 Satır ---')
    display(df.head())
    display(df.tail())
    display(df.sample(5))
    print(df.info())
    display(df.describe().T)
    print(df.columns.tolist())
    print(df.isnull().sum()[df.isnull().sum() > 0])

--- İlk 5 Satır ---


,article,title
0,Data analysis is the process of inspecting and...,Best Books to Learn Data Analysis
1,The performance of a machine learning algorith...,Assumptions of Machine Learning Algorithms
2,You must have seen the news divided into categ...,News Classification with Machine Learning
3,When there are only two classes in a classific...,Multiclass Classification Algorithms in Machin...
4,The Multinomial Naive Bayes is one of the vari...,Multinomial Naive Bayes in Machine Learning


,article,title
29,Many machine learning algorithms can be used t...,Applications of Deep Learning
30,Almost every app or website you visit today sh...,Introduction to Recommendation Systems
31,There are so many algorithms in machine learni...,Use Cases of Different Machine Learning Algori...
32,"In machine learning, the Naive Bayes algorithm...",Naive Bayes Algorithm in Machine Learning
33,Swapping elements of a Python list is very sim...,Swap Items of a Python List


,article,title
12,API stands for Application Programming Interfa...,Best Python Frameworks to Build APIs
25,A scatter plot is one of the most useful ways ...,Animated Scatter Plot using Python
31,There are so many algorithms in machine learni...,Use Cases of Different Machine Learning Algori...
18,Dictionaries are one of the most useful data s...,For Loop Over Keys and Values in a Python Dict...
29,Many machine learning algorithms can be used t...,Applications of Deep Learning


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   article  34 non-null     object
 1   title    34 non-null     object
dtypes: object(2)
memory usage: 676.0+ bytes
None


,count,unique,top,freq
article,34,33,You must have seen the news divided into categ...,2
title,34,33,News Classification with Machine Learning,2


['article', 'title']
Series([], dtype: int64)


In [18]:
# Kolon isimlerini standartlaştır
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace(r'[^a-z0-9_]', '', regex=True)

# 📚 4. Content-Based Recommendation System (TF-IDF & Cosine Similarity)

In [19]:
# TF-IDF Vectorization
articles = data['Article'].tolist()
uni_tfidf = text.TfidfVectorizer(stop_words='english')
uni_matrix = uni_tfidf.fit_transform(articles)

# Cosine Similarity
uni_sim = cosine_similarity(uni_matrix)

# Recommendation Function
def recommend_articles(x):
    # Get top 4 similar articles (excluding itself)
    top_indices = x.argsort()[-5:-1]
    return ', '.join(data['Title'].iloc[top_indices].tolist())

# Apply to DataFrame
data['Recommended Articles'] = [recommend_articles(x) for x in uni_sim]

# Display Result
data.head()

,Article,Title,Recommended Articles
0,Data analysis is the process of inspecting and...,Best Books to Learn Data Analysis,"Introduction to Recommendation Systems, Best B..."
1,The performance of a machine learning algorith...,Assumptions of Machine Learning Algorithms,"Applications of Deep Learning, Best Books to L..."
2,You must have seen the news divided into categ...,News Classification with Machine Learning,"Language Detection with Machine Learning, Appl..."
3,When there are only two classes in a classific...,Multiclass Classification Algorithms in Machin...,"Assumptions of Machine Learning Algorithms, Be..."
4,The Multinomial Naive Bayes is one of the vari...,Multinomial Naive Bayes in Machine Learning,"Assumptions of Machine Learning Algorithms, Me..."


In [20]:
import joblib
# Save the processed data for the App
joblib.dump(data, 'article_data.pkl')
print('Data saved as article_data.pkl')

Data saved as article_data.pkl


## Sonuç ve Değerlendirme

Bu proje, 34 makale üzerinde TF-IDF + Cosine Similarity tabanlı içerik öneri sistemi geliştirmiştir.

| Parametre | Değer |
|-----------|-------|
| Corpus Boyutu | 34 makale |
| Vektörleştirme | TF-IDF (stop_words='english') |
| Benzerlik Metriği | Cosine Similarity |
| Öneri Sayısı | Her makale için top-4 |

**Temel Bulgular:**
- İçerik tabanlı filtreleme, kullanıcı geçmişi gerektirmeden soğuk başlangıç sorununu çözer
- TF-IDF, makine öğrenmesi ve Python odaklı makaleler arasında yüksek benzerlik saptamıştır
- Küçük corpus (34 makale), önerilerin kalitesini sınırlar; daha büyük veri setleriyle sistem ölçeklenebilir
- Daha zengin içerik özellikleri (yazar, etiket, kategori) eklenirse öneri isabeti artacaktır